In [ ]:
# from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, GenerationConfig

In [ ]:
from esm.sdk import client

model = client(model="esm3-small-2024-08", url="https://forge.evolutionaryscale.ai", token="2JGcG8rXj1Ff2FKyQwf5mK")
# model = client(model="esm3-medium-multimer-2024-09", url="https://forge.evolutionaryscale.ai", token="2JGcG8rXj1Ff2FKyQwf5mK")

In [ ]:
model

In [ ]:
sequence = "MKKIAVAAIAVFLAVFASL"

# sequence = "FVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYC/VFPSSVFVPDEWEVSREKITLLRELGQGSFGMVYEGNARDIIKGEAETRVAVKTVNESASLRERIEFLNEASVMKGFTCHHVVRLLGVVSKGQPTLVVMELMAHGDLKSYLRSLRPEAENNPGRPPPTLQEMIQMAAEIADGMAYLNAKKFVHRDLAARNCMVAHDFTVKIGDFGMTRDIYETDYYRKGGKGLLPVRWMAPESLKDGVFTTSSDMWSFGVVLWEITSLAEQPYQGLSNEQVLKFVMDGGYLDQPDNCPERVTDLMRMCWQFNPKMRPTFLEIVNLLKDDLHPSFPEVSFFHSEENK"

In [ ]:
structure_prediction_config = GenerationConfig(
    track="structure",  # We want ESM3 to generate tokens for the structure track
    num_steps=8,
    temperature=0.7,
)

structure_prediction_prompt = ESMProtein(sequence=sequence)

structure_prediction = model.generate(
    structure_prediction_prompt, structure_prediction_config
)

In [ ]:
structure_prediction

In [ ]:
# structure_prediction

In [ ]:
structure_prediction_chain = structure_prediction.to_protein_chain()

In [ ]:
structure_prediction_chain.to_pdb_string()